In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import numpy as np

import __init_path__
import env

from slam.models.relocalization.bovw import BoVW
from slam.utils import resize_image


font = {'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)


csv_path = '/dbstore/datasets/robotics/mlflow/odometry/artifacts/euroc/e43c1a6dffe14121a90a661e84832849/artifacts/predictions/V2_02_medium.csv'
dataset_root = '/dbstore/datasets/Odometry_team/EuRoC_mixed/1'

In [ ]:
df = pd.read_csv(csv_path)
df.head()

In [ ]:
def visualize(df, only_loops, difference_threshold, matches_threshold):
    df = df[df.matches_num >= matches_threshold].reset_index(drop=True)

    frame_total = df.to_index.iloc[-1]
    frame_counter = 1
    
    
    while frame_counter <=  frame_total:  
        matches = df[df['to_index'] == frame_counter]
        frame_counter += 1
        
        if len(matches) > 0:
            
            if only_loops:
                max_difference = 0
                for k in range(len(matches)):
                    difference = matches['to_index'].iloc[k] - matches['from_index'].iloc[k]
                    if difference > max_difference:
                        max_difference = difference
                
                if max_difference < difference_threshold:
                    continue
            
            print(matches[['to_index', 'from_index']])
            
            image = cv2.imread(os.path.join(dataset_root, matches.to_path.iloc[0]))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width, channels_num = image.shape
            small_height = height // 4
            small_width = width // 4
            image = resize_image(image, (small_width, small_height))
            fig, axes = plt.subplots(len(matches) + 1, figsize=(20, 10  * len(matches)))
            axes[0].title.set_text(f'to {matches.to_index.iloc[0]}')
            axes[0].imshow(image / 255)
            for k in range(len(matches)):
                image = cv2.imread(os.path.join(dataset_root, matches.from_path.iloc[k]))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                height, width, channels_num = image.shape
                small_height = height // 4
                small_width = width // 4
                image = resize_image(image, (small_width, small_height))
                axes[k + 1].title.set_text(f'from {matches.from_index.iloc[k]}. Matches {matches.matches_num.iloc[k]}')
                axes[k + 1].imshow(image / 255)
            plt.show()

In [ ]:
is_keyframe = ~np.isnan(df['to_db_index'])
df = df[is_keyframe]

visualize(df, only_loops=True, difference_threshold=100, matches_threshold=40)